In [4]:
%matplotlib inline
import sys
sys.path.append("../src/")
from results import Experiment_Results

import math
import random
import pickle
import operator
import matplotlib
import numpy as np
import pandas as pd
import networkx as nx
from datetime import datetime
import networkx.algorithms.isomorphism as iso
from scipy import stats
from functools import reduce 
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix
from pgmpy.models import BayesianModel
from pgmpy.inference import VariableElimination
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator
from pgmpy.factors.discrete.CPD import TabularCPD
from sklearn.feature_selection import mutual_info_classif
from IPython.display import display, Math
import matplotlib.pyplot as plt
%matplotlib inline 
#pd.set_option('display.max_rows', 200)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 10000)

/opt/conda/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/opt/conda/lib/python3.8/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/opt/conda/lib/python3.8/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [5]:
dataset1 = {'name': "breast-cancer-discretized", \
            'filename': "./breast-cancer-discretized.csv", \
            'folds': [2,5,10], \
            'p_value': [0.05, 0.09, 0.15, 0.20, 0.25, 0.30, 0.35], \
            'refset_size': [10, 40, 100]}

dataset2 = {'name': "breast-cancer-discretized-oversampled", \
            'filename': "./breast-cancer-discretized-oversampled.csv", \
            'folds': [2,5,10], \
            'p_value': [0.05, 0.09, 0.15, 0.20, 0.25, 0.30, 0.35], \
            'refset_size': [10, 40, 100]}
datasets = [dataset1, dataset2]

In [6]:
class Data_Testing:
    def __init__(self, df=None):
        self.df = df
        
    def get_dataframe(self):
        return self.df
        
    def get_kfold_train_test(self, splits=3):
        cv = KFold(n_splits=splits, shuffle=False)
        for train_index, test_index in cv.split(df):
            df_train = df.iloc[train_index,:]
            df_test = df.iloc[test_index,:]
            yield {'df_train': df_train, 'df_test': df_test}
    
    def get_cats(self, col):
        values = np.sort(self.df.loc[:,col].value_counts().index.to_numpy())
        return values
    
    def get_data_from_source(self, data, target, cols=[]):
        ds_target = data.loc[:,target]
        ds_cols = []
        
        for col in cols:
            col1 = data.loc[:,col]
            ds_cols.append(col1)
        ds_ret = {"ds_target": ds_target, "ds_cols": ds_cols} #Return target series and array of col series
    
        return ds_ret

    def get_data(self, target, cols=[]):
        ds_ret = self.get_data_from_source(self.df, target, cols)
    
        return ds_ret

    def g_test(self, samples, target, cols=[]):
        results = None
        data = self.get_data(target, cols)
        con = pd.crosstab(data['ds_target'],data['ds_cols'])
        results = stats.chi2_contingency(con, lambda_="log-likelihood")
        return results[1]

    def chi_test(self, samples, target, cols=[]):
        results = None
        data = self.get_data(target, cols)
        con = pd.crosstab(data['ds_target'],data['ds_cols'])
        results = stats.chi2_contingency(con)
        return results[1]


In [7]:
%%capture capt
class Scatter_Search:
    
    class Metrics:
        def __init__(self, solutions):
            self.solutions = solutions
            
        def sort_solutions(self, metric='accuracy'):
            self.solutions = sorted(self.solutions, key=lambda solution: solution.get_metrics(metric=metric), reverse=True)
            
        def get_top_3_graphs(self):
            self.sort_solutions()
            return self.solutions[0:3]
        
        def get_top_graph_all_metrics(self, metric='accuracy'):
            self.sort_solutions(metric)
            return self.solutions.get_metrics()
        
        def get_number_of_nodes_edges(self, total_graphs=3, metric='accuracy'):
            graph_VE_metrics = []
            self.sort_solutions(metric)
            
            for i in range(0,total_graphs):
                v = self.solutions[i].get_number_of_nodes()
                e = self.solutions[i].get_number_of_edges()
                ve_metric = {'V': v, 'E': e}
                graph_VE_metrics.append(ve_metric)
                
            return graph_VE_metrics
        
        def get_top_graphs_time(self, total_graphs=3, metric='accuracy'):
            graph_times = {}
            self.sort_solutions(metric)
            for i in range(0,total_graphs):
                graph_times['graph-'+str(i+1)] = self.solutions[i].get_bayesian_model().get_total_time().microseconds/1000
            
            return graph_times
        
        def get_confusion_matrix(self, solution):
            cm_metrics = {}
            cm = solution.bn.get_metrics()['cm']
            cm_metrics['TN'] = cm[0][0]
            cm_metrics['FN'] = cm[1][0] 
            cm_metrics['TP'] = cm[1][1]
            cm_metrics['FP'] = cm[0][1]
            cm_metrics['population'] = cm_metrics['TN']+cm_metrics['FN']+cm_metrics['TP']+cm_metrics['FP']
            cm_metrics['NPV'] = cm_metrics['TN']/(cm_metrics['TN']+cm_metrics['FN'])
            cm_metrics['PPV'] = cm_metrics['TP']/(cm_metrics['TP']+cm_metrics['FP'])
            cm_metrics['sensitivity'] = cm_metrics['TP']/(cm_metrics['TP']+cm_metrics['FN'])
            cm_metrics['specificity'] = cm_metrics['TN']/(cm_metrics['TN']+cm_metrics['FP'])
            
            return cm_metrics
            
        def get_top_3_metrix_metrics(self):
            top_3_metrics = []
            top_solutions = self.get_top_3_graphs()
            for solution in top_solutions:
                top_3_metrics.append(self.get_confusion_matrix(solution))
                
            return top_3_metrics
        
    class Reporting:
        def __init__(self, metrics_handler):
            self.mh = metrics_handler
            
        def print_report_1(self):
            print("-----------Top three graphs-----------")
            for solution in self.mh.get_top_3_graphs():
                print(solution.get_metrics())
            print("-----------Top three graphs-----------")
            for metric in self.mh.get_number_of_nodes_edges(total_graphs=3,metric='accuracy'):
                print(metric)
            print("-----------Top three graphs-----------")
            print("---------------Accuracy(ms)-----------")
            print(self.mh.get_top_graphs_time(total_graphs=3, metric='accuracy'))
            print("-----------Top three graphs-----------")
            print("-----------------ROC(ms)--------------")
            print(self.mh.get_top_graphs_time(total_graphs=3, metric='roc'))
            print("-----------Top three graphs-----------")
            print("---------------Recall(ms)-------------")
            print(self.mh.get_top_graphs_time(total_graphs=3, metric='recall'))
            print("-----------Top three graphs-----------")
            print("---------------Precision(ms)----------")
            print(self.mh.get_top_graphs_time(total_graphs=3, metric='precision'))
            print("-----------Top three graphs-----------")
            print("------------CM metrics(ms)------------")
            for cm_metrics in self.mh.get_top_3_metrix_metrics():
                print(cm_metrics)
            
    
    class Solution:
        def __init__(self, graph, p_value):
            self.graph = graph
            self.p_value = p_value
            self.bn = Scatter_Search.Bayesian_Model(graph)
            
        def get_number_of_nodes(self):
            return self.bn.get_bayesian_model().number_of_nodes()
        
        def get_number_of_edges(self):
            return self.bn.get_bayesian_model().number_of_edges()
            
        def get_bayesian_model(self):
            return self.bn
            
        def get_graph(self):
            return self.graph
        
        def get_score(self):
            return self.p_value
        
        def get_training_time(self):
            return self.bn.total_training_time
        
        def get_metrics(self, metric=None):
            if metric is None:
                return self.bn.get_metrics()
            else:
                return self.bn.get_metrics()[metric]
            
        def update_graph_from_bayesian_model(self):
            self.graph = self.bn.create_networkx_graph()
    
    class Solution_Store:
        def __init__(self):
            self.solutions = []
            
        def get_solution_list_size(self):
            return len(self.solutions)
            
        def sort_by_score(self):
            self.solutions.sort(key=lambda sol: sol.get_score())
            
        def get_best_scoring(self):
            self.sort_by_score()
            return self.solutions[0].get_score()
        
        def get_worst_scoring(self):
            self.sort_by_score()
            return self.solutions[len(self.solutions)-1]
        
        def check_solution_is_in_population(self, new_solution):
            is_in_solutions = False
            if len(self.solutions) > 0:
                for solution in self.solutions:
                    nm = iso.categorical_node_match('name', 'name')
                    if nx.is_isomorphic(solution.get_graph(), new_solution.get_graph(), node_match=nm) == True:
                        is_in_solutions = True
                return is_in_solutions
            else:
                return is_in_solutions
        
        def check_solution_is_directed_acyclic_graph(self, new_solution):
            if (nx.is_directed_acyclic_graph(new_solution.get_graph()) == True):
                return True
            else:
                return False
            
        def add_solution_with_checks(self, new_solution):
            if self.check_solution_is_in_population(new_solution) == False:
                if self.check_solution_is_directed_acyclic_graph(new_solution) == True:
                    self.solutions.append(new_solution)
                    return True
                else:
                    pass
            else:
                pass
            return False
        
        def add_solution(self, new_solution):
            self.solutions.append(new_solution)
                
        def get_solution_set(self):
            return self.solutions
        
        def get_metrics_handler(self):
            return Scatter_Search.Metrics(self.solutions)
        
        def save_solutions(self, filename='solutions_store.pkl'):
            fd = open(filename, "wb")
            pickle.dump(self.solutions,fd)
            fd.close()
            
        def score_all_solutions_over_all_data(self):
            for solution in self.solutions:
                solution.get_bayesian_model().score_network_over_all_data()
                
        def limit_solutions(self, solution_limit=10):
            if len(self.solutions) > solution_limit:
                random.shuffle(self.solutions)
                self.solutions = self.solutions[0:solution_limit]
                return self.solutions
        
    class Population(Solution_Store):
        def __init__(self):
            super().__init__()
            
        def get_population_set(self):
            return self.get_solution_set()
        
    class RefSet(Solution_Store):
        def __init__(self):
            super().__init__()
            
        def add_new_solutions(self, solutions, number_of_solutions):
            i=0
            for solution in solutions:
                i=i+1
                if i <= number_of_solutions:
                    self.add_solution_with_checks(solution)
                else:
                    break
                    
        def clean_refset(self, target):
            i=0
            new_solutions = []
            for solution in self.solutions:
                has_target = solution.get_bayesian_model().get_bayesian_model().has_node(target)
                number_of_nodes = len(solution.get_bayesian_model().get_bayesian_model().in_edges(target))
                if ((number_of_nodes > 0) or (has_target is True)):
                    new_solutions.append(solution)
                else:
                    pass
                i=i+1
            self.solutions = new_solutions
                    
        def run_fit_parameters_for_all_models(self, folds=2):
            for solution in self.solutions:
                solution.get_bayesian_model().fit_parameters(k_fold_splits=folds)
                    
        def get_random_solution(self, contains_node=None):
            i = 0
            index_max = (self.get_solution_list_size()-1)
            index = random.randint(0,index_max)
            if contains_node != None:
                while (self.solutions[index].get_graph().has_node(contains_node) == False) and (i < 10):
                    index = random.randint(0,index_max)
                    i=i+1
            if i > 9:
                return None
            else:
                return self.solutions[index]
            
        def find_best_solution(self, metric="accuracy"):
            best_fit_solution = self.solutions[0]
            
            for solution in self.solutions:
                metric1 = best_fit_solution.get_bayesian_model().get_metrics()[metric]
                metric2 = solution.get_bayesian_model().get_metrics()[metric]
                if metric2 > metric1:
                    best_fit_solution = solution
                    
            return best_fit_solution   
        
    class Bayesian_Model:
        def __init__(self, graph, target='Y1'):
            self.bn = BayesianModel()
            self.graph = graph
            self.create_bayesian_network()
            self.metrics = {"accuracy": 0, "roc": 0, "recall": 0, "precision": 0}
            self.target = target
            self.start_training_time = 0
            self.total_training_time = 0
            
        def find_best_graph(self, models, metric):
            current_model = {"bn_model": None, "score": {"accuracy": 0, "roc": 0, "recall": 0, "precision": 0}}
            
            for model in models:
                if model['score'][metric] > current_model['score'][metric]:
                    current_model = model
            
            return current_model
        
        def get_state_names(self, cols=[]):
            state_names_dict = {}
            data = data_testing.get_dataframe()
            
            for col in cols:
                values = list(data[[col]].drop_duplicates().to_numpy().flatten())
                state_names_dict[col] = values
            
            return state_names_dict
        
        def fit_parameters(self, k_fold_splits=2, estimator='mle'):
            bn_models = []
            bn_scores = []
            state_names_dict = {}
            self.set_start_training_time()
            self.trim_to_markov_blanket()
            for data in data_testing.get_kfold_train_test(splits=k_fold_splits):#This has been initalized as gobal in the scatter search init
                bn_model = self.bn.copy()
                if estimator == 'mle':
                    bn_model.fit(data['df_test'][list(bn_model.nodes)], state_names=self.get_state_names(bn_model.nodes))
                elif estimator == 'bayesian':
                    #estimator = BayesianEstimator(bn_model, data)
                    bn_model.fit(data['df_test'][list(bn_model.nodes)], estimator=BayesianEstimator, state_names=self.get_state_names(bn_model.nodes), prior_type='K2')
                score = self.score_network(bn_model, data['df_test'][list(bn_model.nodes)])
                bn_scores.append(score)
                bn_models.append({"bn_model": bn_model, "score": score})
            final_result = self.find_best_graph(bn_models, "accuracy")
            self.bn = final_result['bn_model']
            self.metrics = final_result['score']
            self.set_stop_training_time()
                
        def score_network(self, bn, test_data):
            if self.target in list(bn.nodes):
                td = test_data.drop(self.target, axis=1)
                predictions = bn.predict(td)
                accuracy = accuracy_score(test_data[self.target], predictions)
                try:
                    roc = roc_auc_score(test_data[self.target], predictions)
                except ValueError:
                    roc = 0
                recall = recall_score(test_data[self.target], predictions)
                precision = precision_score(test_data[self.target], predictions)
                cmatrix = confusion_matrix(test_data[self.target], predictions)
                scores = {"accuracy": accuracy, "roc": roc, "recall": recall, \
                          "precision": precision, "cm": cmatrix}
            else:
                scores = {"accuracy": 0, "roc": 0, "recall": 0, "precision": 0, "cm": None}
                
            return scores
        
        def score_network_over_all_data(self):
            nodes = list(self.bn.nodes)
            df = data_testing.get_dataframe()[nodes]
            prediction_df = df.drop(self.target, axis=1)
            predictions = self.bn.predict(prediction_df)
            target_data = df[self.target]
            
            accuracy = accuracy_score(target_data, predictions)
            try:
                roc = roc_auc_score(target_data, predictions)
            except ValueError:
                roc = 0
            recall = recall_score(target_data, predictions)
            precision = precision_score(target_data, predictions)
            cmatrix = confusion_matrix(target_data, predictions)
            self.metrics = {"accuracy": accuracy, "roc": roc, "recall": recall, \
                            "precision": precision, "cm": cmatrix}
            
        def trim_to_markov_blanket(self):
            G = BayesianModel()
            nodes = self.bn.get_markov_blanket(self.target)
            nodes.append(self.target)
            for edge in self.bn.in_edges:
                if (edge[0] in nodes) or (edge[1] in nodes):
                    G.add_edge(edge[0], edge[1])
                    
            for edge in self.bn.out_edges:
                if (edge[0] in nodes) or (edge[1] in nodes):
                    G.add_edge(edge[0], edge[1])
                    
            self.bn = G
            
        def create_bayesian_network(self):
            for edge in self.graph.edges:
                self.bn.add_edge(edge[0], edge[1])
            #self.fit_parameters()
            
        def create_networkx_graph(self):
            G = nx.DiGraph()
            for edge in self.bn.edges():
                G.add_edge(edge[0], edge[1])
                
            return G
            
        def get_target_edge_nodes(self, target):
            out_nodes = set()
            edges = self.bn.edges
            
            for edge in edges:
                if edge[0] == target:
                    out_nodes.add(edge[1])
                    
            return out_nodes
        
        def has_target(self, target):
            edges = self.bn.edges(target)
            
            if len(edges) > 0:
                return True
            else:
                return False
        
        def get_bayesian_model(self):
            return self.bn
        
        def get_metrics(self):
            return self.metrics
        
        def set_start_training_time(self):
            self.start_training_time = datetime.now()
            
        def set_stop_training_time(self):
            self.total_training_time = datetime.now() - self.start_training_time
            
        def get_total_time(self):
            return self.total_training_time

    ######################################################
        
    def __init__(self, df, target='Y1', x_cols=[], y_cols=[]):
        global data_testing
        data_testing = Data_Testing(df)
        self.data_testing = data_testing
        self.df = df
        self.target = target
        self.population = Scatter_Search.Population()
        self.refset = Scatter_Search.RefSet()
    
    def build_graph_from_cols(self, target, cols):
        G = nx.DiGraph()
        G.add_node(target, name=target)
        for col in cols:
            G.add_edge(col, target)
            
        return G
    
    def diversification_from_target(self, alpha):
        new_pop_solutions = []
        cols = (self.df.iloc[0:1].drop(self.target, axis=1)).columns
        for col in cols:
            p_value = self.data_testing.chi_test(len(df.index), self.target, [col])
            if p_value < alpha:
                G = nx.DiGraph()
                G.add_node(col, name=col)
                G.add_node(self.target, name=self.target)
                G.add_edge(col, self.target)
                new_pop_solutions.append(Scatter_Search.Solution(G, p_value))
        for new_solution in new_pop_solutions:
            self.population.add_solution_with_checks(new_solution)
            
    def diversification_from_comparing(self, alpha):
        new_pop_solutions = []
        cols = (self.df.iloc[0:1].drop(self.target, axis=1)).columns
        for col1 in cols:
            for col2 in cols:
                if col1 != col2:
                    p_value1 = self.data_testing.chi_test(len(df.index), col1, [col2])
                    if p_value1 < alpha:
                        G1 = self.build_graph_from_cols(col1, [col2])
                        new_pop_solutions.append(Scatter_Search.Solution(G1, p_value1))
        for new_solution in new_pop_solutions:
            self.population.add_solution_with_checks(new_solution)
            
    
    def diversification(self, alpha=0.05):
        self.diversification_from_target(alpha)
        self.diversification_from_comparing(alpha)
            
    def best_scoring_solutions(self):
        target_solutions = Scatter_Search.Solution_Store()
        
        for solution in ss.population.get_population_set():
            G = solution.get_graph()
            add_graph = True
            for out_data in G.edges.data():
                if out_data[1] in [self.target]:
                    target_solutions.add_solution_with_checks(solution)

        target_solutions.sort_by_score()
        
        return target_solutions
    
    def diverse_solutions(self): #Find solutions not directly connected to Y
        population_solutions = ss.population.get_population_set()
        nodes = set()
        diverse_solutions = Scatter_Search.Solution_Store()
        
        for sol in population_solutions:
            for edge_tuple in sol.get_graph().edges:
                if self.target == edge_tuple[1]:
                    nodes.add(edge_tuple[0])
        for sol in population_solutions:
            if sol.get_graph().has_node(self.target) == False:
                for out_data in sol.get_graph().edges:
                    if out_data[0] in list(nodes):
                        diverse_solutions.add_solution_with_checks(sol)
                        
        diverse_solutions.sort_by_score()
                        
        return diverse_solutions
                    
    def populate_refset(self, limit_number_of_solutions=10):
        number_of_solutions_from_each = round(limit_number_of_solutions/2)
        self.refset.add_new_solutions(self.best_scoring_solutions().get_solution_set(), number_of_solutions_from_each)
        self.refset.add_new_solutions(self.diverse_solutions().get_solution_set(), number_of_solutions_from_each)
        
    def combine_solutions(self, solution1, solution2):
        new_solution = None
        G = nx.compose(solution1.get_graph(),solution2.get_graph())
        if nx.is_directed_acyclic_graph(G) == True:
            new_solution = Scatter_Search.Solution(G, 0)
        else:
            print("Error graph is not acyclic....")
            pass
        
        return new_solution
    
    def subset_generation_method(self):
        solution1 = self.refset.get_random_solution()
        solution2 = solution1
        while(solution1 == solution2 or solution2 is None):
            solution2 = self.refset.get_random_solution(contains_node=self.target)
        if (solution1 != None and solution2 != None):
            return self.combine_solutions(solution1, solution2)
        else:
            print("New solution not found, none type returned")
            return None
    
    def run_scatter_search(self, fold, p_value, refset_size, experiment_name):
        last_refset_update=0
        number_of_iterations=0
        startTime = datetime.now()
        ss.diversification(alpha=p_value)
        self.populate_refset()
        print("Refset size: " + str(self.refset.get_solution_list_size()))
        
        while(last_refset_update < 20 and number_of_iterations < 500):
            soultion = self.subset_generation_method()
            if soultion == None:
                print("No solutions found...")
                break
            if self.refset.add_solution_with_checks(soultion) == True:
                last_refset_update = 0
                #nx.draw_networkx(soultion.get_graph())
                plt.show()
            else:
                last_refset_update = last_refset_update+1
            number_of_iterations = number_of_iterations+1
        
        print("Number of iterations: " + str(number_of_iterations))
        print("Last refset update count: " + str(last_refset_update))
        print("Refset size: " + str(self.refset.get_solution_list_size()))
        self.refset.clean_refset('Y1')
        print("Refset size: " + str(self.refset.get_solution_list_size()))
        before_refset_limit = self.refset.get_solution_list_size()
        self.refset.limit_solutions(solution_limit=refset_size)
        print("Refset size: " + str(self.refset.get_solution_list_size()))
        self.refset.run_fit_parameters_for_all_models(fold)
        self.refset.score_all_solutions_over_all_data()
        solution = self.refset.find_best_solution()
        print(solution.get_bayesian_model().get_metrics())
        print("Number of nodes: " + str(solution.get_number_of_nodes()))
        self.refset.save_solutions()
        mh = self.refset.get_metrics_handler()
        solutions = mh.get_top_3_graphs()
        for solution in solutions:
            print("Accuracy: " + str(solution.get_metrics(metric="accuracy")))
        print("--------\n\n\n\n--------")
        print("Total runtime: " + str(datetime.now() - startTime))
        solutions = mh.get_top_3_graphs()
        for solution in solutions:
            bnm = solution.get_bayesian_model()
            print(bnm.get_metrics())
            print("Nodes: " + str(len(bnm.get_bayesian_model().nodes)) + " Edges: " + str(len(bnm.get_bayesian_model().edges)))
            nx.draw_networkx(bnm.get_bayesian_model())
            plt.show()
        report = self.Reporting(mh)
        report.print_report_1()
        
        erc = Experiment_Results()
        erc.load_df('../results/pd_results_dataframe.pkl')
        i=0
        for graph in mh.get_top_3_graphs():
            i=i+1
            results_row = {'dt': datetime.now(),
                                'experiment_name': experiment_name,
                                'model_name': 'scatter search',
                                'graph': i,
                                'p_value': p_value,
                                'refset_size': refset_size,
                                'before_limit_refset_size': before_refset_limit,
                                'vertices': graph.get_number_of_nodes(),
                                'edge': graph.get_number_of_edges(),
                                'runtime': graph.get_bayesian_model().get_total_time().microseconds/1000,
                                'folds': fold,
                                'samples': df.iloc[:,0].count(),
                                'population': mh.get_confusion_matrix(graph)['population'],
                                'accuracy': graph.get_bayesian_model().get_metrics()['accuracy'],
                                'ROC': graph.get_bayesian_model().get_metrics()['roc'],
                                'recall': graph.get_bayesian_model().get_metrics()['recall'],
                                'precision': graph.get_bayesian_model().get_metrics()['precision'],
                                'TN': mh.get_confusion_matrix(graph)['TN'],
                                'FN': mh.get_confusion_matrix(graph)['FN'],
                                'TP': mh.get_confusion_matrix(graph)['TP'],
                                'FP': mh.get_confusion_matrix(graph)['FP'],
                                'NPV': mh.get_confusion_matrix(graph)['NPV'],
                                'PPV': mh.get_confusion_matrix(graph)['PPV'],
                                'sensitivity': mh.get_confusion_matrix(graph)['sensitivity'],
                                'specificity': mh.get_confusion_matrix(graph)['specificity'],
                                'PRC': 0}
            erc.add_row_as_dict(results_row)
            
        erc.save_df('../results/pd_results_dataframe.pkl')
        return erc

for dataset in datasets:
    df = pd.read_csv(dataset['filename'])
    df = df.sample(frac=1)
    for fold in dataset['folds']:
        for p_value in dataset['p_value']:
            for refset_size in dataset['refset_size']:
                ss = Scatter_Search(df)
                erc = ss.run_scatter_search(fold, p_value, refset_size, dataset['name'])
             

KeyboardInterrupt: 

In [7]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

In [30]:
'''mh = ss.refset.get_metrics_handler()
mh.get_top_3_metrix_metrics()
mh.get_top_3_graphs()[0].get_number_of_nodes()
mh.get_top_3_graphs()[0].get_number_of_edges()
mh.get_top_3_graphs()[0].get_number_of_edges()
mh.get_top_3_graphs()[0].get_bayesian_model().get_total_time().microseconds/1000
mh.get_confusion_matrix(mh.get_top_3_graphs()[0])
mh.get_top_3_graphs()[0].get_bayesian_model().get_metrics()['accuracy']
mh.get_top_3_graphs()[0].get_bayesian_model().get_metrics()['roc']
mh.get_top_3_graphs()[0].get_bayesian_model().get_metrics()['recall']
mh.get_top_3_graphs()[0].get_bayesian_model().get_metrics()['precision']'''

"mh = ss.refset.get_metrics_handler()\nmh.get_top_3_metrix_metrics()\nmh.get_top_3_graphs()[0].get_number_of_nodes()\nmh.get_top_3_graphs()[0].get_number_of_edges()\nmh.get_top_3_graphs()[0].get_number_of_edges()\nmh.get_top_3_graphs()[0].get_bayesian_model().get_total_time().microseconds/1000\nmh.get_confusion_matrix(mh.get_top_3_graphs()[0])\nmh.get_top_3_graphs()[0].get_bayesian_model().get_metrics()['accuracy']\nmh.get_top_3_graphs()[0].get_bayesian_model().get_metrics()['roc']\nmh.get_top_3_graphs()[0].get_bayesian_model().get_metrics()['recall']\nmh.get_top_3_graphs()[0].get_bayesian_model().get_metrics()['precision']"

In [31]:
'''mh = ss.refset.get_metrics_handler()
mh.get_confusion_matrix(mh.get_top_3_graphs()[0])'''

'mh = ss.refset.get_metrics_handler()\nmh.get_confusion_matrix(mh.get_top_3_graphs()[0])'

In [32]:
'''mh.get_confusion_matrix(mh.get_top_3_graphs()[0])'''

'mh.get_confusion_matrix(mh.get_top_3_graphs()[0])'

In [38]:
erc.get_df()

,dt,experiment_name,model_name,graph,vertices,edge,runtime,folds,samples,population,accuracy,ROC,recall,precision,TN,FN,TP,FP,NPV,PPV,sensitivity,specificity,PRC
0,2021-08-22 21:31:13.989901,acute-inflammations-diagnosis-discretized,MLPClassifier,0,NaN,NaN,2290.821,2,120,120,1.000000,1.00,1.00,1.000000,70.0,0.0,50.0,0.0,1.000000,1.000000,1.00,1.0,0.0
1,2021-08-22 21:31:14.002741,acute-inflammations-diagnosis-discretized,KNeighborsClassifier,0,NaN,NaN,1035.399,2,120,120,1.000000,1.00,1.00,1.000000,70.0,0.0,50.0,0.0,1.000000,1.000000,1.00,1.0,0.0
2,2021-08-22 21:31:14.014473,acute-inflammations-diagnosis-discretized,SVC-Linear,0,NaN,NaN,993.763,2,120,120,0.908333,0.91,0.92,0.867925,63.0,4.0,46.0,7.0,0.940299,0.867925,0.92,0.9,0.0
3,2021-08-22 21:31:14.025909,acute-inflammations-diagnosis-discretized,SVC,0,NaN,NaN,997.047,2,120,120,1.000000,1.00,1.00,1.000000,70.0,0.0,50.0,0.0,1.000000,1.000000,1.00,1.0,0.0
4,2021-08-22 21:31:14.037273,acute-inflammations-diagnosis-discretized,GaussianProcessClassifier,0,NaN,NaN,879.632,2,120,120,1.000000,1.00,1.00,1.000000,70.0,0.0,50.0,0.0,1.000000,1.000000,1.00,1.0,0.0
5,2021-08-22 21:31:14.048593,acute-inflammations-diagnosis-discretized,DecisionTreeClassifier,0,NaN,NaN,25.304,2,120,120,1.000000,1.00,1.00,1.000000,70.0,0.0,50.0,0.0,1.000000,1.000000,1.00,1.0,0.0
6,2021-08-22 21:31:14.059891,acute-inflammations-diagnosis-discretized,RandomForestClassifier,0,NaN,NaN,109.352,2,120,120,1.000000,1.00,1.00,1.000000,70.0,0.0,50.0,0.0,1.000000,1.000000,1.00,1.0,0.0
7,2021-08-22 21:31:14.071194,acute-inflammations-diagnosis-discretized,AdaBoostClassifier,0,NaN,NaN,235.476,2,120,120,1.000000,1.00,1.00,1.000000,70.0,0.0,50.0,0.0,1.000000,1.000000,1.00,1.0,0.0
8,2021-08-22 21:31:14.082516,acute-inflammations-diagnosis-discretized,GaussianNB,0,NaN,NaN,18.848,2,120,120,0.966667,0.96,0.92,1.000000,70.0,4.0,46.0,0.0,0.945946,1.000000,0.92,1.0,0.0
9,2021-08-22 21:31:14.094022,acute-inflammations-diagnosis-discretized,QuadraticDiscriminantAnalysis,0,NaN,NaN,23.169,2,120,120,1.000000,1.00,1.00,1.000000,70.0,0.0,50.0,0.0,1.000000,1.000000,1.00,1.0,0.0
